<a href="https://colab.research.google.com/github/crystalloide/Notebooks-SQL-serie-1/blob/main/s01_Transaction_(Begin%2C_Commit%2C_Rollback).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## s01-Transaction (Begin, Commit, Rollback)

Generally the SQLite is in auto-commit mode that means SQLite automatically starts a transaction for each command, process and commit the transaction changes automatically to database. However, we still can disable auto-commit mode and use the following three commands to control these transactions to maintain data consistency and to handle database errors based on our requirements:

- BEGIN – start the transaction;
- COMMIT – commit the transaction that means all the changes saved to database;
- ROLLBACK – rollback the complete transaction.

Transactional control commands are only used with commands INSERT, UPDATE, and DELETE. They cannot be used while creating tables or dropping them because these operations are automatically committed in the database.

In this notebook, we will have a test on the command of ***DELETE***, only because we already practiced UPDATE and INSERT before. In addition, we will use the ***[sqlite3](https://docs.python.org/2/library/sqlite3.html)*** package because ipython_sql does not support transactions.

The ***DELETE*** is a very dangerous command so it is better to firstly back up your database before applying it. 

In [1]:
import sqlite3 as lite
import sys

In [2]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


### 1. Connection à la database demo.db3

In [3]:
con = lite.connect('/content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3')


### 2. Make a test table

This time, we still use the table of watershed_yearly to make a test table just as we did in the previous notebook.

In [4]:
with con:
    cur = con.cursor() 
    
    sql = """
    DROP TABLE  IF EXISTS watershed_yearly_bk;
    CREATE TABLE watershed_yearly_bk AS SELECT YR, PREC_mm FROM watershed_yearly
    """
    
    cur.executescript(sql)    

Make some NULLs

In [5]:
with con:
    cur = con.cursor()
    sql = """
    UPDATE watershed_yearly_bk
    SET PREC_mm = NULL
    WHERE
    PREC_mm < 850.0
    """    
    cur.executescript(sql)    

Have a quick view

In [7]:
with con:      
    cur = con.cursor()    
    cur.execute("SELECT * FROM watershed_yearly_bk")

    rows = cur.fetchall()

    for row in rows:
        print (row)

(1981, 895.6051025390625)
(1982, 884.670654296875)
(1983, None)
(1984, 867.5743408203125)
(1985, None)
(1986, None)
(1987, 1007.8944702148438)
(1988, 895.8466186523438)
(1989, 930.10546875)
(1990, None)
(1991, 984.4703369140625)
(1992, 907.9463500976562)
(1993, 1057.7733154296875)
(1994, None)
(1995, None)
(1996, None)
(1997, None)
(1998, None)
(1999, None)
(2000, None)
(2001, None)
(2002, None)
(2003, None)
(2004, None)
(2005, 855.0092163085938)
(2006, None)
(2007, None)
(2008, None)
(2009, 1040.9012451171875)
(2010, 905.66845703125)


### 2. ROLLBACK a  DELETE transaction

We'd like to drop/delete all rows with NULL values.

In [8]:
con.isolation_level = None
cur = con.cursor()
cur.execute("BEGIN")
sql = """
      DELETE FROM watershed_yearly_bk WHERE PREC_mm IS NULL
      """
cur = con.execute(sql)
cur.execute("ROLLBACK")

Now check the changes and you can find nothing happend.

In [ ]:
cur = con.cursor()    
cur.execute("SELECT * FROM watershed_yearly_bk")

rows = cur.fetchall()

for row in rows:
    print row

### 3. COMMIT a DELETE transaction

In [ ]:
cur = con.cursor()
cur.execute("BEGIN")
sql = """
      DELETE FROM watershed_yearly_bk WHERE PREC_mm IS NULL
      """
cur = con.execute(sql)
cur.execute("COMMIT")

Now check the changes and you can find the rows with NULLs have been deleted.

In [ ]:
cur = con.cursor()    
cur.execute("SELECT * FROM watershed_yearly_bk")

rows = cur.fetchall()

for row in rows:
    print row

### 4. Close the db connection

In [ ]:
con.close()

### Summary and References

Using ***sqlite3*** is not that panic because we can easily wrap the query sentences in the previous notebook into strings. This notebook also show there are other ways accessing SQLite database. If you are intested in it, you can try [SQLAlchemy](https://docs.sqlalchemy.org/en/latest/dialects/sqlite.html), which is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

https://www.tutorialspoint.com/sqlite/sqlite_transactions.htm

https://www.tutlane.com/tutorial/sqlite/sqlite-transactions-begin-commit-rollback.

http://zetcode.com/db/sqlitepythontutorial/

https://docs.sqlalchemy.org/en/latest/dialects/sqlite.html

https://docs.python.org/2/library/sqlite3.html